In [24]:
import pandas as pd
import numpy as np
import random

import unicodedata
from rapidfuzz import fuzz

import re
import unicodedata
from collections import Counter

import time

In [ ]:

df = pd.read_csv("names_dataset.csv")

# Estructura
print(df.head())
print(df.shape)

# Unicidad
print("Nombres únicos:", df['Full Name'].nunique())

# Longitud de strings
print(df['Full Name'].str.len().describe())



   ID             Full Name
0   1         María Sánchez
1   2          Marta Alonso
2   3       Javier González
3   4    Carmen López López
4   5  Isabel Moreno Moreno
(5000, 2)
Nombres únicos: 3016
count    5000.000000
mean       17.314000
std         4.644967
min         8.000000
25%        13.000000
50%        17.000000
75%        21.000000
max        33.000000
Name: Full Name, dtype: float64


El dataset provisto (names_dataset.csv) contiene:

* 5.000 registros

* 2 columnas: ID y Full Name

* Nombres completos en español, con:

* Acentos

* Nombres y apellidos compuestos

* Cantidad variable de tokens

* Ausencia de estructura explícita (nombre / apellido)

Por lo tanto, el problema corresponde a matching textual, no a análisis semántico.

En primer lugar comprobamos si es conveniente realizar una separación previa de los datos en nombre y apellido dado que comunmente los sistemas de información tienen esta estructura. Además la ponderación mayor al apellido podría ayudar a enfocar más el algoritmo, sin embargo dado que no hay una estructura confiable (el dataset no viene separado previamente ni con un separador) separar nombre y apellido genera un peor score. 

# Alternativas evaluadas
## A. Comparación sobre el nombre completo

Este enfoque consiste en trabajar directamente sobre el nombre completo tal como aparece en el dataset. Previamente, los textos se normalizan (conversión a minúsculas y eliminación de acentos) para evitar diferencias superficiales.
La comparación se realiza sobre el string completo utilizando técnicas de similaridad basadas en tokens y distancia de edición, lo que permite capturar variaciones de orden, errores de tipeo y omisiones parciales sin asumir una estructura fija del nombre.

## B. Separación heurística de nombre y apellido

En esta alternativa se intenta dividir automáticamente el nombre completo en dos partes: nombre y apellido, asumiendo que el último token corresponde al apellido y el resto al nombre.
Luego, se calcula la similitud de cada componente por separado y se obtiene un score final ponderado, asignando mayor peso al apellido. Este enfoque busca priorizar coincidencias de apellido, aunque depende fuertemente de una heurística que no siempre se cumple en nombres reales.

# Experimento empírico

Se realizaron pruebas con consultas realistas contra el dataset, comparando los resultados obtenidos con ambos enfoques. El objetivo fue evaluar no solo el score numérico, sino también la calidad del ranking y la coherencia de las coincidencias devueltas.

# Resultados observados

El enfoque basado en el nombre completo mostró resultados más precisos y consistentes en la mayoría de los casos.
Por el contrario, la separación heurística introdujo varios problemas: aumentó la cantidad de falsos positivos, sobreponderó coincidencias parciales de apellido y perdió información relevante en nombres compuestos. En múltiples escenarios, esto provocó que el resultado correcto quedara peor posicionado en el ranking.


| Query             | Mejor match (Full) | Score    | Mejor match (Split)      | Score    |
| ----------------- | ------------------ | -------- | ------------------------ | -------- |
| Maria Sanchez     | María Sánchez      | 100      | María Sánchez            | 100      |
| Juan Carlos Perez | Juan Pérez         | 100      | Juan Pérez               | 100      |
| Luis Gonzalez     | Luis González      | 100      | **Luis Martín González** | 100      |
| Ana Lopez         | Ana Sánchez López  | 100      | **Ana Jiménez López**    | 100      |
| Pedro Ramirez     | Pedro Ruiz         | **78.2** | Pedro Álvarez            | **74.3** |

In [4]:
%pip install rapidfuzz

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\pbonafe\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
# Cargar dataset
df = pd.read_csv("names_dataset.csv")

def normalize(text: str) -> str:
    text = text.lower()
    text = unicodedata.normalize("NFKD", text)
    text = "".join(c for c in text if not unicodedata.combining(c))
    return " ".join(text.split())

df["normalized_name"] = df["Full Name"].apply(normalize)

#enfoque A
def similarity_full(query: str, candidate: str) -> float:
    return fuzz.token_set_ratio(query, candidate)

#enfoque B
def split_name(full_name: str):
    tokens = full_name.split()
    if len(tokens) == 1:
        return tokens[0], ""
    return " ".join(tokens[:-1]), tokens[-1]

def similarity_split(query: str, candidate: str, w_name=0.4, w_surname=0.6) -> float:
    q_name, q_surname = split_name(query)
    c_name, c_surname = split_name(candidate)

    score_name = fuzz.token_set_ratio(q_name, c_name) if q_name and c_name else 0
    score_surname = fuzz.token_set_ratio(q_surname, c_surname) if q_surname and c_surname else 0

    return score_name * w_name + score_surname * w_surname


#prueba con datos sintéticos
queries = [
    "Maria Sanchez",
    "Juan Carlos Perez",
    "Luis Gonzalez",
    "Ana Lopez",
    "Pedro Ramirez"
]

results = []

for q in queries:
    q_norm = normalize(q)

    for _, row in df.iterrows():
        full_score = similarity_full(q_norm, row["normalized_name"])
        split_score = similarity_split(q_norm, row["normalized_name"])

        results.append({
            "query": q,
            "id": row["ID"],
            "candidate": row["Full Name"],
            "full_score": round(full_score, 2),
            "split_score": round(split_score, 2)
        })

#Tabla comparativa final
results_df = pd.DataFrame(results)





In [9]:
required_cols = {"query", "candidate", "full_score", "split_score"}
missing = required_cols - set(results_df.columns)
if missing:
    raise ValueError(f"results_df no tiene las columnas requeridas: {sorted(missing)}")

# 1) Mejor match por enfoque FULL (máximo full_score por query; desempate estable)
best_full = (
    results_df
    .sort_values(["query", "full_score", "id"], ascending=[True, False, True], kind="mergesort")
    .drop_duplicates(subset=["query"], keep="first")
    .rename(columns={"candidate": "candidate_full", "full_score": "full_score"})
    .loc[:, ["query", "candidate_full", "full_score"]]
)

# 2) Mejor match por enfoque SPLIT (máximo split_score por query; desempate estable)
best_split = (
    results_df
    .sort_values(["query", "split_score", "id"], ascending=[True, False, True], kind="mergesort")
    .drop_duplicates(subset=["query"], keep="first")
    .rename(columns={"candidate": "candidate_split", "split_score": "split_score"})
    .loc[:, ["query", "candidate_split", "split_score"]]
)

# 3) Tabla comparativa final (lado a lado)
comparison = best_full.merge(best_split, on="query", how="inner")

comparison


,query,candidate_full,full_score,candidate_split,split_score
0,Ana Lopez,Ana López López,100.00,Ana López López,100.00
1,Juan Carlos Perez,Carlos Pérez,100.00,Carlos Pérez,100.00
2,Luis Gonzalez,Dr. Luis González,100.00,Dr. Luis González,100.00
3,Maria Sanchez,María Sánchez,100.00,María Sánchez,100.00
4,Pedro Ramirez,Pedro Ruiz,78.26,Pedro Romero Álvarez,74.29


# Conclusión

Separar nombre y apellido sin contar con una estructura confiable en los datos introduce heurísticas frágiles que terminan degradando la precisión del matching.
En este dataset en particular, no existe una garantía semántica de que el último token represente correctamente el apellido ni de que los tokens restantes correspondan al nombre. Asumir esa estructura genera errores sistemáticos difíciles de corregir.

El enfoque más robusto, explicable y defendible consiste en trabajar sobre el nombre completo, aplicando tokenización y técnicas de comparación textual tolerantes a errores, sin forzar divisiones artificiales.

# Estrategia final adoptada

La estrategia elegida evita la creación de campos artificiales de nombre y apellido y se basa en la normalización del texto y el uso de métricas de similaridad que combinan comparación por tokens y distancia de edición.
Los resultados se filtran según un umbral configurable y se ordenan de mayor a menor similitud.

Este enfoque reduce falsos positivos, no depende de reglas arbitrarias, es fácil de explicar en una entrevista técnica y escala adecuadamente para el tamaño del dataset analizado.

# Data mining & Data Cleaning

Analizamos el dataset con múltiples ciclos de LLMs: GPT - Gemini para identificar los patrones subyacentes en los datos y poder estandarizar la base de consulta. De esta manera la api puede trabajar sobre datos normalizados. Luego probamos la performance del algoritmo que usamos en la api matcheando la base original vs la base estandarizada para definir cual utilizar.

# Análisis exploratorio del dataset de nombres

Archivo: `names_dataset.csv`

Objetivo: entender **calidad**, **duplicados**, **variantes por errores humanos** y definir una **estandarización**  para usar como base de consulta.


In [ ]:
CSV_PATH = "names_dataset.csv"  # ajustá si lo movés
df = pd.read_csv(CSV_PATH)

display(df.head())
print("shape:", df.shape)
print("columns:", list(df.columns))


,ID,Full Name
0,1,María Sánchez
1,2,Marta Alonso
2,3,Javier González
3,4,Carmen López López
4,5,Isabel Moreno Moreno


shape: (5000, 2)
columns: ['ID', 'Full Name']


## 1) Chequeos básicos (IDs, nulos, duplicados exactos)

- Verificamos si `ID` es único.
- Cuántos nombres son nulos.
- Cuántos duplicados exactos hay en `Full Name`.


In [3]:
# Calidad básica
basic = {
    "rows": len(df),
    "unique_id": df["ID"].nunique(),
    "null_full_name": int(df["Full Name"].isna().sum()),
    "unique_full_name_raw": df["Full Name"].astype(str).nunique(),
    "exact_duplicates_raw": int(len(df) - df["Full Name"].astype(str).nunique()),
}
basic


{'rows': 5000,
 'unique_id': 5000,
 'null_full_name': 0,
 'unique_full_name_raw': 3016,
 'exact_duplicates_raw': 1984}

## 2) Distribución de longitudes y cantidad de tokens

Nos ayuda a entender si hay nombres compuestos, dobles apellidos, etc.


In [4]:
df["full_name_raw"] = df["Full Name"].astype(str)
df["len_chars"] = df["full_name_raw"].str.len()
df["n_tokens_raw"] = df["full_name_raw"].str.strip().str.split().map(len)

stats = {
    "len_p50": float(df["len_chars"].median()),
    "len_p95": float(df["len_chars"].quantile(0.95)),
    "tokens_distribution": df["n_tokens_raw"].value_counts().sort_index().to_dict(),
}
stats


{'len_p50': 17.0,
 'len_p95': 25.0,
 'tokens_distribution': {2: 2228, 3: 2550, 4: 222}}

## 3) Detección de caracteres sospechosos / ruido

Buscamos:
- Títulos (`Dr.`, `Lic.`, etc.)
- Puntuación / símbolos (`$`, `+`, paréntesis)
- Doble espacio / espacios al inicio o final

Esto suele ser típico de *data entry* humano.


In [5]:
# caracteres fuera de letras/espacios/'/-
pattern_susp = re.compile(r"[^A-Za-zÁÉÍÓÚÜÑáéíóúüñ\s'\-\.]")
susp = df[df["full_name_raw"].str.contains(pattern_susp, regex=True, na=False)]

# espacios
df["has_double_space"] = df["full_name_raw"].str.contains(r"\s{2,}", regex=True, na=False)
df["has_leading_trailing_space"] = df["Full Name"].astype(str).str.match(r"^\s|\s$", na=False)

# títulos (token inicial terminando en punto)
df["first_token"] = df["full_name_raw"].str.strip().str.split().str[0]
df["has_title_prefix"] = df["first_token"].str.endswith(".", na=False)

report_noise = {
    "rows_with_suspicious_chars": int(len(susp)),
    "rows_with_double_space": int(df["has_double_space"].sum()),
    "rows_with_leading_or_trailing_space": int(df["has_leading_trailing_space"].sum()),
    "rows_with_title_prefix": int(df["has_title_prefix"].sum()),
    "top_title_prefixes": df[df["has_title_prefix"]]["first_token"].value_counts().head(15).to_dict(),
}
report_noise


{'rows_with_suspicious_chars': 123,
 'rows_with_double_space': 136,
 'rows_with_leading_or_trailing_space': 0,
 'rows_with_title_prefix': 487,
 'top_title_prefixes': {'Lic.': 93,
  'Col.': 89,
  'Dr.': 87,
  'Sr.': 80,
  'Sra.': 75,
  'Mg.': 63}}

aparecen muchos nombres con caracteres como títulos que harán fallar luego el algoritmo de matching

## 4) Normalización base (acentos, mayúsculas, espacios)

Definimos una normalización *suave* (no elimina títulos) para medir cuánto baja la cardinalidad por diferencias superficiales.


In [6]:
def strip_accents(s: str) -> str:
    return "".join(
        c for c in unicodedata.normalize("NFKD", s)
        if not unicodedata.combining(c)
    )

def normalize_soft(raw: str) -> str:
    if raw is None or (isinstance(raw, float) and np.isnan(raw)):
        return ""
    s = str(raw).strip()
    s = re.sub(r"\s+", " ", s)
    s = strip_accents(s).lower()
    # mantenemos letras, espacios, apóstrofe, guion; removemos el resto
    s = re.sub(r"[^a-z\s'\-]", "", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["name_norm_soft"] = df["full_name_raw"].map(normalize_soft)

soft_stats = {
    "unique_soft": int(df["name_norm_soft"].nunique()),
    "duplicates_soft": int(len(df) - df["name_norm_soft"].nunique()),
}
soft_stats


{'unique_soft': 2872, 'duplicates_soft': 2128}

## 5) Normalización estricta (remueve títulos comunes + símbolos)

Como el dataset muestra títulos y símbolos, definimos una normalización *estricta* que:
- elimina prefijos tipo `Dr.` / `Lic.` / `Sr.` / `Sra.` / `Mg.` / `Col.`
- elimina dígitos y símbolos
- mantiene `'` y `-` por apellidos compuestos

Esto nos da una base estandarizada más robusta para indexar.


In [7]:
TITLE_PREFIXES = {"dr", "lic", "sr", "sra", "mg", "col"}

def normalize_strict(raw: str) -> str:
    if raw is None or (isinstance(raw, float) and np.isnan(raw)):
        return ""
    s = str(raw).strip()
    s = re.sub(r"\s+", " ", s)

    tokens = s.split(" ")
    if tokens and tokens[0].rstrip(".").lower() in TITLE_PREFIXES:
        tokens = tokens[1:]
    s = " ".join(tokens)

    s = strip_accents(s).lower()
    # reemplazamos separadores raros por espacio
    s = re.sub(r"[^\w\s'\-]", " ", s)
    # removemos dígitos / underscores
    s = re.sub(r"[_\d]+", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["name_norm_strict"] = df["full_name_raw"].map(normalize_strict)

strict_stats = {
    "unique_strict": int(df["name_norm_strict"].nunique()),
    "duplicates_strict": int(len(df) - df["name_norm_strict"].nunique()),
}
strict_stats


{'unique_strict': 2619, 'duplicates_strict': 2381}

## 6) Variantes por mismo nombre normalizado

Mide cuántas representaciones distintas (raw) terminan en el mismo valor normalizado (indicador de errores humanos / variantes ortográficas).


In [8]:
variants = (
    df.groupby("name_norm_strict")["full_name_raw"]
      .nunique()
      .sort_values(ascending=False)
)

# cuántos normalizados tienen más de 1 variante raw
multi_variant_count = int((variants > 1).sum())

# ejemplos
examples = variants[variants > 1].head(20)

multi_variant_count, examples


(292,
 name_norm_strict
 javier jimenez      5
 jose moreno         5
 pablo jimenez       5
 sofia martinez      4
 marta moreno        4
 marta gonzalez      4
 marta ruiz          4
 carlos ruiz         4
 juan fernandez      4
 lucia gutierrez     4
 sofia alonso        4
 pilar diaz          4
 javier gonzalez     4
 sofia alvarez       4
 carlos rodriguez    4
 marta hernandez     4
 sofia hernandez     4
 sofia gomez         4
 lucia ruiz          4
 jose rodriguez      4
 Name: full_name_raw, dtype: int64)

## 7) Duplicados frecuentes

Top nombres más repetidos (raw vs normalizado). Sirve para entender si hay muchos nombres comunes y qué impacto tiene en el matching.


In [9]:
top_raw = df["full_name_raw"].value_counts().head(15)
top_strict = df["name_norm_strict"].value_counts().head(15)

display(top_raw.to_frame("count_raw"))
display(top_strict.to_frame("count_norm_strict"))


,count_raw
full_name_raw,
Marta Sánchez,12
Pilar Sánchez,12
Fernando Rodríguez,11
María Martínez,11
Sofía Martín,11
Fernando Gutiérrez,11
María Jiménez,11
Javier Muñoz,10
Francisco Alonso,10


,count_norm_strict
name_norm_strict,
marta sanchez,13
jose moreno,13
juan romero,12
sofia martin,12
sofia alonso,12
pablo martin,12
pilar sanchez,12
francisco gutierrez,11
maria jimenez,11


## 8) Hallazgos y decisiones (para el informe)

### Hallazgos principales
- El dataset contiene ruido típico de carga humana (títulos con punto, símbolos y puntuación).
- Hay duplicados exactos y duplicados que aparecen solo después de normalizar (acentos/mayúsculas/espacios/puntuación).
- No existe estructura confiable para separar nombre/apellido (nombres compuestos y cantidad de tokens variable).

### Decisiones de estandarización (base de consulta)
1. **No separar nombre/apellido**: se trabaja con el nombre completo.
2. Crear `name_norm_strict` como clave estándar para indexación:
   - lowercase
   - sin acentos
   - colapsar espacios
   - remover símbolos/dígitos
   - remover prefijos de títulos comunes
   - preservar `-` y `'`
3. Mantener el `Full Name` original para mostrar al usuario (explicabilidad).



In [12]:
# Export opcional: dataset estandarizado para inspección / reutilización
#out_path = "./API/data/clean/names_dataset_standardized.csv"
out_path = "names_dataset_standardized.csv"
df_out = df[["ID", "Full Name", "name_norm_soft", "name_norm_strict"]].copy()
df_out.to_csv(out_path, index=False)
out_path


'names_dataset_standardized.csv'

In [ ]:
### Próximo paso
- Aplicar la misma normalización al input del usuario (y opcionalmente remover títulos si el usuario los incluye).
- Construir el motor de búsqueda (candidate generation + scoring) sobre `name_norm_strict`.

## Pruebas

Vamos a tomar 10 nombres al azar del csv, le generaremos ruido simulando errores humanos, luego ejecutamos el mismo matching que usa la API y comparamos los resultados matcheando contra el dataset original vs el dataset normalizado. Además incluimos una variante estandarizado + dedupe (la que suele mejorar más la performance).

Esperamos que mejore:
* Performance (latencia/QPS): mejorarás solo si la “base normalizada” te permite comparar contra menos strings (ej: deduplicar por clave normalizada) o evitar cómputo repetido (precomputar name_normalized, tokens, n-grams, etc.).

* Calidad (ranking): puede mejorar si la normalización reduce ruido (“Dr.”, símbolos, dobles espacios, acentos, etc.).

10 nombres al azar + ruido

In [15]:
# -------------------------
# 10 nombres al azar (reproducible)
# -------------------------
SAMPLE_SEED = 42
sample_names = df["Full Name"].sample(10, random_state=SAMPLE_SEED).tolist()

# -------------------------
# Simular errores humanos (typos, acentos, símbolos, etc.)
# -------------------------
random.seed(SAMPLE_SEED)

def make_noisy(q: str) -> str:
    s = str(q)

    # 50%: quitar acentos
    if random.random() < 0.5:
        s = strip_accents(s)

    # 50%: borrar un carácter (typo)
    if len(s) > 6 and random.random() < 0.5:
        i = random.randint(1, len(s) - 2)
        s = s[:i] + s[i+1:]

    # 30%: agregar puntuación extra
    if random.random() < 0.3:
        s = s + " !!"

    return s

queries = []
for clean in sample_names:
    noisy = make_noisy(clean)
    queries.append({"clean_name": clean, "query_noisy": noisy, "query_norm": normalize_strict(noisy)})

qdf = pd.DataFrame(queries)
qdf


,clean_name,query_noisy,query_norm
0,Ana García González,Ana Garcí González !!,ana garci gonzalez
1,Sofía Jiménez,Sofia Jimnez !!,sofia jimnez
2,Javier Álvarez Hernández,Javier lvarez Hernandez !!,javier lvarez hernandez
3,Francisco Fernández,Francisco Fernández,francisco fernandez
4,Dr. Lucía Rodríguez,Dr. Lucía Rodríuez,lucia rodriuez
5,Pedro García Jiménez,Pedro arcía Jiménez,pedro arcia jimenez
6,Elena Sánchez Gómez,Elena Sanchz Gomez !!,elena sanchz gomez
7,Sofía Fernández,Sofia ernandez,sofia ernandez
8,Ca@rmen Hernández,Ca@rmen Hernández,ca rmen hernandez
9,Carmen Hernández Hernández,Carmen Hernández Hrnández !!,carmen hernandez hrnandez


Matching (API-like) + benchmark original vs estandarizado

In [16]:
def match_against_repo(
    query: str,
    repo_ids: list,
    repo_norm_names: list,
    threshold: float = 70,
    limit: int = 10,
    w_token: float = 0.65
):
    qn = normalize_strict(query)
    hits = []

    for _id, cand in zip(repo_ids, repo_norm_names):
        t = fuzz.token_set_ratio(qn, cand)
        e = fuzz.ratio(qn, cand)
        s = w_token * t + (1.0 - w_token) * e

        if s >= threshold:
            hits.append((_id, s, t, e))

    # orden determinista (igual que pediste en la API)
    hits.sort(key=lambda x: (x[1], x[2], x[3]), reverse=True)

    return hits[:limit]


3.2 Repos a comparar

Repo “original”: usa el CSV original, pero normalizamos en memoria (como hace la API al construir repo)

Repo “standardized”: usa name_strict ya precomputado del CSV estandarizado

Repo “standardized + dedupe”: elimina duplicados por strict_key (suele mejorar performance)

In [20]:

CSV_PATH = "names_dataset.csv"

# -------------------------
# Normalización (misma idea que venimos usando)
# -------------------------
TITLE_PAT = re.compile(r"^(dr|dra|sr|sra|srta|ing|lic|prof)\.?\s+", re.IGNORECASE)
NON_LETTER = re.compile(r"[^a-zA-Z\s]")
MULTISPACE = re.compile(r"\s+")

def strip_accents(s: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFKD", s) if not unicodedata.combining(c))

def normalize_strict(name: str) -> str:
    s = str(name).strip().lower()
    s = strip_accents(s)
    s = TITLE_PAT.sub("", s)          # elimina títulos al inicio
    s = NON_LETTER.sub(" ", s)        # elimina símbolos/puntuación
    s = MULTISPACE.sub(" ", s).strip()
    return s

# -------------------------
# Cargar dataset
# -------------------------
df = pd.read_csv(CSV_PATH)
assert "ID" in df.columns and "Full Name" in df.columns

df_std = df.copy()
df_std["name_strict"] = df_std["Full Name"].map(normalize_strict)

# Clave para detectar duplicados semánticos (misma “persona” para el motor)
df_std["strict_key"] = df_std["name_strict"]
df_std["is_strict_dup"] = df_std.duplicated("strict_key", keep="first")
df_std["strict_group_size"] = df_std.groupby("strict_key")["ID"].transform("size")

print("Rows:", len(df_std))
print("Unique strict_key:", df_std["strict_key"].nunique())
print("Duplicados por strict_key:", int(df_std["is_strict_dup"].sum()))

# Guardar dataset estandarizado (si querés persistirlo)
OUT_STD = "names_dataset_standardized.csv"
df_std.to_csv(OUT_STD, index=False)
print("Saved:", OUT_STD)


Rows: 5000
Unique strict_key: 2717
Duplicados por strict_key: 2283
Saved: names_dataset_standardized.csv


In [22]:
ids_all = df_std["ID"].tolist()

# A) "Original" (normalizo desde Full Name)
repo_original_norm = df["Full Name"].astype(str).map(normalize_strict).tolist()

# B) "Standardized" (ya trae name_strict)
repo_standardized_norm = df_std["name_strict"].tolist()

# C) "Standardized + dedupe" (menos comparaciones)
df_dedup = df_std[~df_std["is_strict_dup"]].copy()
ids_dedup = df_dedup["ID"].tolist()
repo_dedup_norm = df_dedup["name_strict"].tolist()

print("repo_original:", len(ids_all))
print("repo_standardized:", len(ids_all))
print("repo_dedup:", len(ids_dedup))


repo_original: 5000
repo_standardized: 5000
repo_dedup: 2717


La eliminación de duplicados en la base a comparar aqui representará una mejora en el benchmark marginal pero en una escala de 50M de datos es sumamente relevante.

In [25]:
def benchmark(repo_ids, repo_norm_names, queries, runs=30):
    lats = []
    hitcounts = []

    for q in queries:
        for _ in range(runs):
            t0 = time.perf_counter()
            hits = match_against_repo(q, repo_ids, repo_norm_names, threshold=70, limit=10, w_token=0.65)
            lats.append((time.perf_counter() - t0) * 1000.0)
            hitcounts.append(len(hits))

    lats = np.array(lats)
    return {
        "runs": int(len(lats)),
        "mean_ms": float(lats.mean()),
        "p50_ms": float(np.percentile(lats, 50)),
        "p95_ms": float(np.percentile(lats, 95)),
        "min_ms": float(lats.min()),
        "max_ms": float(lats.max()),
        "avg_hits": float(np.mean(hitcounts)),
    }

queries_noisy = qdf["query_noisy"].tolist()

bench_original = benchmark(ids_all, repo_original_norm, queries_noisy, runs=20)
bench_standard = benchmark(ids_all, repo_standardized_norm, queries_noisy, runs=20)
bench_dedup = benchmark(ids_dedup, repo_dedup_norm, queries_noisy, runs=20)

pd.DataFrame([
    {"repo": "original", **bench_original},
    {"repo": "standardized", **bench_standard},
    {"repo": "standardized+dedupe", **bench_dedup},
])


,repo,runs,mean_ms,p50_ms,p95_ms,min_ms,max_ms,avg_hits
0,original,200,8.218177,7.6870,10.77084,5.8791,35.0053,10.0
1,standardized,200,8.570966,8.5748,9.96902,7.2256,14.2805,10.0
2,standardized+dedupe,200,5.447388,5.2325,7.55574,4.0855,9.0630,10.0


3.4 Comparación rápida de resultados (top-5) por query

In [26]:
def topk_table(repo_name, repo_ids, repo_norm_names):
    rows = []
    for q in queries_noisy:
        hits = match_against_repo(q, repo_ids, repo_norm_names, threshold=70, limit=5, w_token=0.65)
        for rank, (id_, s, t, e) in enumerate(hits, start=1):
            rows.append({
                "repo": repo_name,
                "query": q,
                "rank": rank,
                "id": int(id_),
                "similarity": round(s, 2),
                "token_score": round(t, 2),
                "edit_score": round(e, 2),
            })
    return pd.DataFrame(rows)

out = pd.concat([
    topk_table("original", ids_all, repo_original_norm),
    topk_table("standardized", ids_all, repo_standardized_norm),
    topk_table("standardized+dedupe", ids_dedup, repo_dedup_norm),
], ignore_index=True)

out.sort_values(["query", "repo", "rank"]).head(30)


,repo,query,rank,id,similarity,token_score,edit_score
0,original,Ana Garcí González !!,1,1502,97.30,97.30,97.30
1,original,Ana Garcí González !!,2,1022,93.00,100.00,80.00
2,original,Ana Garcí González !!,3,1172,93.00,100.00,80.00
3,original,Ana Garcí González !!,4,1594,93.00,100.00,80.00
4,original,Ana Garcí González !!,5,2277,93.00,100.00,80.00
50,standardized,Ana Garcí González !!,1,1502,97.30,97.30,97.30
51,standardized,Ana Garcí González !!,2,1022,93.00,100.00,80.00
52,standardized,Ana Garcí González !!,3,1172,93.00,100.00,80.00
53,standardized,Ana Garcí González !!,4,1594,93.00,100.00,80.00
54,standardized,Ana Garcí González !!,5,2277,93.00,100.00,80.00


Acá vemos que en el primer caso en la comparación original vs standarized la estandarizacion no cambia el ranquing, estamos normalizando la query antes de la consulta y el dataset no tiene un ruido estructural grave por lo que el cambio no es sustancial. En algunos casos como el 2 y 3 como antes había muchos registros casi identicos ahora al eliminar los duplicados semánticos se liberan slots del top-k por lo que aparecen candidatos distintos pero razonables, no es que empeora el modelo sino que compara contra más variedad. Esto es deseable en producción donde se hidrata por ID y se aplica un re-ranking posterior. 

Con respecto al benchmark, la estandarización comun no solo no reduce la latencia sino que es marginalmente peor por ruido estadístico. Sin embargo la deduplicación semántica si genera una mejora sustancial 35% menos de latencia media y un p95 mucho mas estable con menos picos máximos de latencia. Lo cual es sustancialmente mejor porque genera menos candidatos reales, menos comparaciones, menor trabajo por query y mejor predictivilidad. Esto claramente será preferible con 50 millones de datos.
En este sentido, la estandarización se justifica no como un mecanismo de mejora directa del matching, sino como un habilitador clave para escalar el sistema de forma eficiente y predecible.

En producción lo que deberíamos hacer es:

✔ Mantener:

+ Source of Truth con datos originales

+ IDs intactos

✔ Construir:

+ tabla/índice de búsqueda estandarizado

+ clave normalizada (strict_key)

+ estructura deduplicada para candidate retrieval

✔ En la API:

+ buscar contra el índice deduplicado

+ devolver IDs

+ hidratar desde SoT

+ opcionalmente expandir a todos los IDs del grupo
